# Import Sources
This example shows how to import voltage and current sources. In this example, we are going to

- Download an example board
- Create a configuration file
  - Add a voltage source between two nets
  - Add a current source between two pins
  - Add a current source between two pin groups
  - Add a current source between two coordinates
  - Add a current source to the nearest pin
  - Add distributed sources
- Import the configuration file

## Import the required packages

In [1]:
import json
from pathlib import Path
import tempfile

from ansys.aedt.core.downloads import download_file

from pyedb import Edb

AEDT_VERSION = "2024.2"
NG_MODE = False


Download the example PCB data.

In [2]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", destination=temp_folder.name)

## Load example layout

In [3]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.34.3


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database ANSYS-HSD_V1.aedb Opened in 2024.2


PyAEDT INFO: Cell main Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


## Create an empty dictionary to host all configurations

In [4]:
cfg = dict()

## Add a voltage source between two nets

Keywords

- **name**. Name of the voltage source.
- **Reference_designator**. Reference designator of the component.
- **type**. Type of the source. Supported types are 'voltage', 'current'
- **positive_terminal**. Supported types are 'net', 'pin', 'pin_group', 'coordinates'
- **negative_terminal**. Supported types are 'net', 'pin', 'pin_group', 'coordinates',
'nearest_pin'

In [5]:
voltage_source = {
    "name": "V_SOURCE_5V",
    "reference_designator": "U4",
    "type": "voltage",
    "magnitude": 1,
    "positive_terminal": {"net": "5V"},
    "negative_terminal": {"net": "GND"},
}

## Add a current source between two pins

In [6]:
current_source_1 = {
    "name": "I_CURRENT_1A",
    "reference_designator": "J5",
    "type": "current",
    "magnitude": 10,
    "positive_terminal": {"pin": "15"},
    "negative_terminal": {"pin": "14"},
}

## Add a current source between two pin groups

In [7]:
pin_groups = [
    {"name": "IC2_5V", "reference_designator": "IC2", "pins": ["8"]},
    {"name": "IC2_GND", "reference_designator": "IC2", "net": "GND"},
]

In [8]:
current_source_2 = {
    "name": "CURRENT_SOURCE_2",
    "type": "current",
    "positive_terminal": {"pin_group": "IC2_5V"},
    "negative_terminal": {"pin_group": "IC2_GND"},
}

## Add a current source between two coordinates

Keywords

- **layer**. Layer on which the terminal is placed
- **point**. XY coordinate the terminal is placed
- **net**. Name of the net the terminal is placed on

In [9]:
current_source_3 = {
    "name": "CURRENT_SOURCE_3",
    "type": "current",
    "positive_terminal": {"coordinates": {"layer": "1_Top", "point": ["116mm", "41mm"], "net": "5V"}},
    "negative_terminal": {"coordinates": {"layer": "Inner1(GND1)", "point": ["116mm", "41mm"], "net": "GND"}},
}

## Add a current source reference to the nearest pin

Keywords

- **reference_net**. Name of the reference net
- **search_radius**. Reference pin search radius in meter

In [10]:
current_source_4 = {
    "name": "CURRENT_SOURCE_4",
    "reference_designator": "J5",
    "type": "current",
    "positive_terminal": {"pin": "16"},
    "negative_terminal": {"nearest_pin": {"reference_net": "GND", "search_radius": 5e-3}},
}

## Add distributed current sources

Keywords

- **distributed**. Whether to create distributed sources. When set to True, ports are created per pin

In [11]:
sources_distributed = {
    "name": "DISTRIBUTED",
    "reference_designator": "U2",
    "type": "current",
    "distributed": True,
    "positive_terminal": {"net": "5V"},
    "negative_terminal": {"net": "GND"},
}

## Add setups in configuration

In [12]:
cfg["pin_groups"] = pin_groups
cfg["sources"] = [
    voltage_source,
    current_source_1,
    current_source_2,
    current_source_3,
    current_source_4,
    sources_distributed,
]

## Write configuration into as json file

In [13]:
file_json = Path(temp_folder.name) / "edb_configuration.json"
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

## Import configuration into example layout

In [14]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

True

## Review

In [15]:
edbapp.siwave.sources

{'V_SOURCE_5V': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x17513b41e10>,
 'I_CURRENT_1A': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x1755a67f910>,
 'CURRENT_SOURCE_2': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x1755a67fbe0>,
 'CURRENT_SOURCE_3': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x17517262a40>,
 'CURRENT_SOURCE_4': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x17517260b50>,
 'DISTRIBUTED_U2_5V_39': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x17517262350>,
 'DISTRIBUTED_U2_5V_40': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x175172604c0>,
 'DISTRIBUTED_U2_5V_41': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x17517262a10>,
 'DISTRIBUTED_U2_5V_42': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x17517262890>,
 'DISTRIBUTED_U2_5V_43': <pyedb.dotnet.edb_core.edb_data.ports.ExcitationSources at 0x17517262860>,
 'DISTRIBUTED_U2_5V_44': <pye

## Save and close Edb
The temporary folder will be deleted once the execution of this script is finished. Replace **edbapp.save()** with
**edbapp.save_as("C:/example.aedb")** to keep the example project.

In [16]:
edbapp.save()
edbapp.close()

True